## 0. Import


In [1]:
import argparse
import glob
import json
import os
from pathlib import Path

import numpy as np
import torch
import yaml
from tqdm import tqdm

from models.experimental import attempt_load
from utils.datasets import create_dataloader
from utils.general import coco80_to_coco91_class, check_dataset, check_file, check_img_size, box_iou, \
    non_max_suppression, scale_coords, xyxy2xywh, xywh2xyxy, clip_coords, set_logging, increment_path
from utils.loss import compute_loss
from utils.metrics import ap_per_class
from utils.plots import plot_images, output_to_target
from utils.torch_utils import select_device, time_synchronized

import wandb

import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.patches as patches

# 1. Parameter


In [2]:
parser_helmet = argparse.ArgumentParser(prog='test.py')
parser_helmet.add_argument('--single-cls', action='store_true', help='treat as single-class dataset')
parser_helmet.add_argument('--aug', type=str, default='n', help='in test py img augmentation')
parser_helmet.add_argument('--mode', type=str, default='helmet', help='which label to train with (both/helmet/alone)')
opt_helmet = parser_helmet.parse_args(args=[])

parser_alone = argparse.ArgumentParser(prog='test.py')
parser_alone.add_argument('--single-cls', action='store_true', help='treat as single-class dataset')
parser_alone.add_argument('--aug', type=str, default='n', help='in test py img augmentation')
parser_alone.add_argument('--mode', type=str, default='alone', help='which label to train with (both/helmet/alone)')
opt_alone = parser_alone.parse_args(args=[])


In [3]:
data_helmet = 'data/helmet.yaml'
data_alone = 'data/alone.yaml'
weights_helmet = 'runs/train/helmet/weights/best_ap50.pt'
weights_alone = 'runs/train/alone/weights/best_ap50.pt'

aug = 'n'
batch_size = 32
imgsz = 512
augment = False
verbose=False
model=None
dataloader=None
save_dir=Path('')
save_txt=False
save_conf=False
plots=True
log_imgs=0
device= 'cpu'

task = 'val' #todo, ['val','test']
conf_thres = 0.001 #todo
iou_thres = 0.65 #todo

# 2. Inference

In [4]:
set_logging()
device = select_device(device, batch_size=batch_size)

save_dir = '/opt/ml/final_project/final-project-level3-cv-01/code/runs/merge_test'
model_helmet = attempt_load(weights_helmet, map_location=device)
model_alone = attempt_load(weights_alone, map_location=device)
imgsz = check_img_size(imgsz,s=model_helmet.stride.max())

model_helmet.eval()
model_alone.eval()

with open(data_helmet) as f:
    data_helmet = yaml.load(f,Loader=yaml.FullLoader)
check_dataset(data_helmet)

with open(data_alone) as f:
    data_alone = yaml.load(f,Loader=yaml.FullLoader)
check_dataset(data_alone)

img = torch.zeros((1, 3, imgsz, imgsz), device=device)
_ = model_helmet(img)  # run once
_ = model_alone(img)  # run once

path_helmet = data_helmet['test'] if task == 'test' else data_helmet['val']  # path to val/test images
path_alone = data_alone['test'] if task == 'test' else data_alone['val']  # path to val/test images

dataloader = create_dataloader(path_helmet, imgsz, batch_size, model_helmet.stride.max(), opt_helmet, pad=0.5, rect=True)[0]

Using torch 1.10.0+cu102 CPU



Fusing layers... 


/opt/conda/envs/lightweight/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 751 layers, 150960248 parameters, 0 gradients, 232.2 GFLOPS


Fusing layers... 


Model Summary: 751 layers, 150960248 parameters, 0 gradients, 232.2 GFLOPS
Scanning labels /opt/ml/final_project/data/val/labels_helmet.cache3 (245 found, 0 missing, 0 empty, 0 duplicate, for 245 images): 245it [00:00, 12111.55it/s]
/opt/conda/envs/lightweight/lib/python3.7/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
def box_iou(box1, box2):
    # https://github.com/pytorch/vision/blob/master/torchvision/ops/boxes.py
    """
    Return intersection-over-union (Jaccard index) of boxes.
    Both sets of boxes are expected to be in (x1, y1, x2, y2) format.
    Arguments:
        box1 (Tensor[N, 4])
        box2 (Tensor[M, 4])
    Returns:
        iou (Tensor[N, M]): the NxM matrix containing the pairwise
            IoU values for every element in boxes1 and boxes2
    """

    def box_area(box):
        # box = 4xn
        return (box[2] - box[0]) * (box[3] - box[1])

    area1 = box_area(box1.T)
    area2 = box_area(box2.T)

    # inter(N,M) = (rb(N,M,2) - lt(N,M,2)).clamp(0).prod(2)
    inter = (torch.min(box1[:, None, 2:], box2[:, 2:]) - torch.max(box1[:, None, :2], box2[:, :2])).clamp(0).prod(2)
    return inter / (area1[:, None] + area2 - inter)  # iou = inter / (area1 + area2 - inter)

def remove_overlap(iou, priority = 'alone'):
    if priority == 'alone':
        # 가로 먼저 max
        iou *= (iou == iou.max(dim=1, keepdim=True)[0])
        iou *= (iou == iou.max(dim=0, keepdim=True)[0])
    else :
        iou *= (iou == iou.max(dim=1, keepdim=True)[0])
        iou *= (iou == iou.max(dim=0, keepdim=True)[0])
    
    return iou
        
def both_class(helmet, alone):
    return 2*alone + helmet

In [6]:
# targets [n,6] - (img_num,cls,x,y,w,h)
# paths [32,1]
# img [nb,3,w,h]
for batch_i, (img, targets, paths, shapes) in enumerate(tqdm(dataloader)):
    img = img.to(device, non_blocking=True)
    img = img.float()
    img /= 255.0
    targets.to(device)
    
    nb, _, height, width = img.shape
    whwh = torch.Tensor([width,height, width, height]).to(device)

    with torch.no_grad():
        #run model - [32,11475,7] (x,y,w,h,obj_conf,cls_conf for each class)
        inf_out_helmet, train_out_helmet = model_helmet(img, augment=augment)
        inf_out_alone, train_out_alone = model_alone(img, augment=augment)
    
        #run nms - [32,-,6] (x1,y1,x2,y2,conf,cls)
        output_helmet = non_max_suppression(inf_out_helmet, conf_thres=conf_thres, iou_thres=iou_thres, merge=False, classes=None, agnostic=True)
        output_alone = non_max_suppression(inf_out_alone, conf_thres=conf_thres, iou_thres=iou_thres)

    output = []
    # ith image in a batch
    for image_i in range(nb):
        # prediction for ith image (x1,y1,x2,y2,conf,cls)
        pred_helmet = output_helmet[image_i]
        pred_alone = output_alone[image_i]
        
        # helmet or alone에 box 없는 경우
        if len(pred_helmet) == 0 or len(pred_alone) == 0 :
            continue
        

        #pred_both
        iou = box_iou(pred_helmet[:,:4],pred_alone[:,:4])
        iou = remove_overlap(iou)
        i,j = (iou > iou_thres).nonzero(as_tuple=False).T

        pred_helmet = pred_helmet[i]
        pred_alone = pred_alone[j]

        box_n = len(pred_helmet)
        pred_both = torch.zeros_like(pred_helmet)

        output.append(pred_both)
        

        for index in range(box_n):
            # todo - box merge는 나중에 & obj_conf로 하는 것이 더 정확
            if pred_helmet[index][4] > pred_alone[index][4]:
                pred_both[index][:4] = pred_helmet[index][:4]
            else:
                pred_both[index][:4] = pred_alone[index][:4]
            pred_both[index][4] = pred_helmet[index][4] * pred_alone[index][4]
            pred_both[index][5] = both_class(pred_helmet[index][5],pred_alone[index][5])
        
        path = Path(paths[image_i])
        


        # (cls,x,y,w,h)
        labels = targets[targets[:,0]==image_i, 1:]
        gn = torch.tensor(shapes[image_i][0])[[1,0,1,0]]
        x = pred_both.clone()
        x[:,:4] = scale_coords(img[image_i].shape[1:],x[:,:4],shapes[image_i][0],shapes[image_i][1])
        # for *xyxy, conf, cls in x:
        #     xywh = (xyxy2xywh(torch.tensor(xyxy).view(1,4))/gn).view(-1).tolist()
        #     line = (cls, *xywh, conf)
        #     with open(save_dir / 'labels' / (path.stem+'.txt'),'a') as f:
        #         f.write(('%g ' * len(line)).rstrip() % line + '\n')

        break
    break
            


  0%|          | 0/8 [00:45<?, ?it/s]


In [9]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

False

In [7]:
output

[tensor([[6.13896e+01, 7.66511e+01, 1.39537e+02, 2.10082e+02, 9.47482e-01, 1.00000e+00],
         [3.89434e+02, 6.74094e+01, 5.07789e+02, 2.07215e+02, 3.01251e-02, 1.00000e+00],
         [2.77112e+02, 5.97449e+01, 2.94687e+02, 1.12453e+02, 4.62524e-03, 1.00000e+00]])]

# 3. Non-Maximum Suppression

In [6]:
prediction = inf_out_helmet
conf_thres=0.1
iou_thres=0.6
merge=False
classes=None
agnostic=False

nc = prediction[0].shape[1]-5
xc = prediction[...,4]>0.1

min_wh, max_wh =2,4096
max_det = 300
time_limit = 10.0
redundant = True
multi_label = False #확인필요
output = [torch.zeros(0,6)]*prediction.shape[0]

In [7]:
prediction.shape

torch.Size([32, 11475, 7])

x => [x,y,w,h,conf,cls1,cls2]

In [8]:
for xi, x in enumerate(prediction):
    x = x[xc[xi]]    
    x[:,5:] *= x[:,4:5]
    box = xywh2xyxy(x[:,:4])    

    i,j = ((x[:,5:] > conf_thres).nonzero(as_tuple=False).T)
    x = torch.cat((box[i], x[i, j + 5, None], j[:, None].float()), 1)
   
    c = x[:,5:6] * (0 if agnostic else max_wh)
    boxes, scores = x[:,:4] + c, x[:,4]
    
    i = torch.ops.torchvision.nms(boxes,scores,iou_thres)

    
    break


In [9]:
print(boxes.shape)

torch.Size([69, 4])


In [21]:
def box_iou(box1, box2):
    # https://github.com/pytorch/vision/blob/master/torchvision/ops/boxes.py
    """
    Return intersection-over-union (Jaccard index) of boxes.
    Both sets of boxes are expected to be in (x1, y1, x2, y2) format.
    Arguments:
        box1 (Tensor[N, 4])
        box2 (Tensor[M, 4])
    Returns:
        iou (Tensor[N, M]): the NxM matrix containing the pairwise
            IoU values for every element in boxes1 and boxes2
    """

    def box_area(box):
        # box = 4xn
        return (box[2] - box[0]) * (box[3] - box[1])

    area1 = box_area(box1.T)
    area2 = box_area(box2.T)

    # inter(N,M) = (rb(N,M,2) - lt(N,M,2)).clamp(0).prod(2)
    inter = (torch.min(box1[:, None, 2:], box2[:, 2:]) - torch.max(box1[:, None, :2], box2[:, :2])).clamp(0).prod(2)
    return inter / (area1[:, None] + area2 - inter)  # iou = inter / (area1 + area2 - inter)


In [22]:
i

tensor([19, 36,  1], device='cuda:0')

In [ ]:
iou = box_iou(boxes[i],boxes) > iou_thres
weights = iou * scores # 겹치는 애들의 score만 담겨있음

In [51]:
weights

tensor([[0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.96911, 0.31675, 0.00000, 0.00000, 0.96161, 0.25876, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.97065, 0.32645, 0.00000, 0.00000, 0.96185, 0.26204, 0.75962, 0.95285, 0.00000, 0.00000, 0.23226, 0.95426, 0.00000, 0.00000, 0.76906,
         0.95700, 0.00000, 0.00000, 0.25917, 0.95554, 0.00000, 0.00000, 0.75373, 0.93551, 0.00000, 0.00000, 0.20580, 0.93377, 0.00000, 0.00000, 0.70325, 0.00000, 0.00000, 0.89013, 0.89927, 0.00000, 0.00000, 0.38096, 0.58364, 0.00000, 0.00000, 0.76206, 0.79237, 0.00000, 0.00000, 0.22858, 0.00000, 0.00000, 0.00000,
         0.00000],
        [0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.45703, 0.66789, 0.00000, 0.00000, 0.64723, 0.68575, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.45689, 0.69521, 0.00000, 0.00000, 0.65722, 0.70071, 0.00000, 0.00000, 0.00000, 0.00000, 0.92003, 0.94481, 0.00000, 0.00000, 0.31033, 0.61730, 0.00000,
         0.00000, 0.92084, 0.94831, 

In [50]:
x[:,:4]

tensor([[276.80698,  59.43233, 294.75516, 112.06213],
        [277.11203,  59.74487, 294.68680, 112.45279],
        [276.62393,  59.90397, 294.43344, 112.36182],
        [277.20520,  59.71339, 294.88123, 112.11556],
        [277.27734,  59.95576, 294.67615, 111.97173],
        [276.88263,  60.30616, 294.48621, 112.68191],
        [386.30835,  67.42755, 505.37646, 206.13791],
        [392.62170,  66.76973, 506.28363, 205.94713],
        [ 60.61163,  77.43890, 139.28905, 210.87970],
        [ 61.04763,  77.34317, 140.26407, 209.71539],
        [387.94220,  65.83866, 505.37506, 210.02353],
        [391.16357,  65.17422, 506.88971, 209.14368],
        [ 60.84024,  77.69925, 138.88235, 210.05832],
        [ 60.98352,  78.15411, 140.14357, 210.83667],
        [277.08905,  59.85069, 294.90881, 112.04035],
        [277.27585,  60.20796, 294.73459, 112.41620],
        [276.75940,  60.25511, 294.81714, 112.24501],
        [386.35049,  68.29788, 505.68698, 205.17816],
        [393.13177,  67.0339

In [54]:
x[i,:4] = torch.mm(weights, x[:,:4]) / weights.sum(1, keepdim=True)


In [57]:
print(iou)
print(iou.sum(1))

tensor([[False, False, False, False, False, False, False, False,  True,  True, False, False,  True,  True, False, False, False, False, False,  True,  True, False, False,  True,  True,  True,  True, False, False,  True,  True, False, False,  True,  True, False, False,  True,  True, False, False,  True,  True, False,
         False,  True,  True, False, False,  True, False, False,  True,  True, False, False,  True,  True, False, False,  True,  True, False, False,  True, False, False, False, False],
        [False, False, False, False, False, False,  True,  True, False, False,  True,  True, False, False, False, False, False,  True,  True, False, False,  True,  True, False, False, False, False,  True,  True, False, False,  True,  True, False, False,  True,  True, False, False,  True,  True, False, False,  True,
          True, False, False,  True,  True, False,  True,  True, False, False,  True,  True, False, False,  True,  True, False, False,  True,  True, False,  True,  True,  True,  Tru